# Mapping charter schools

Authors: 
- Harshayu Girase, URAP research apprentice, Computer Science major, UC Berkeley
- Serena Jiang, URAP research apprentice, Computer Science major, UC Berkeley
- Jaren Haber, PhD Candidate, Sociology, UC Berkeley

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from shapely.geometry import Point
import geopandas as gpd
import csv
from geopandas.tools import sjoin
import folium
from folium import plugins
from folium.plugins import MarkerCluster
#import pyepsg
from folium import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
import fiona
import pysal as ps

/opt/conda/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [2]:
import os
os.getcwd()

'/home/jovyan/work/geospatial/scripts'

In [3]:
# Define data file paths
dir_prefix = '/home/jovyan/work/'

charterfile = dir_prefix + 'data_management/data/old/charters_parsed_03-04_no-text_SMALL.csv'
districtfile = dir_prefix + 'data_management/data/ACS_2016_sd-merged_SMALL.csv'
publicfile = dir_prefix + 'pub_schools_merged_2015.csv'

shapefileuni = dir_prefix + 'geospatial/shapefiles/US_sd_uni_2016.shp'
shapefileelm = dir_prefix + 'geospatial/shapefiles/US_sd_elm_2016.shp'
shapefilesec = dir_prefix + 'geospatial/shapefiles/US_sd_sec_2016.shp'
#on top of shapefile, districtfile is the districts. publicfile school data. chartefile non used.

In [4]:
# Create merged geometric DF using district, elementary, secondary school district shapefiles
unishape = gpd.read_file(shapefileuni) # Create shapefile DF with only geometric data (for general use)
elmshape = gpd.read_file(shapefileelm)
secshape = gpd.read_file(shapefilesec)

#'sdshape' has polygons of the districts.
sdshape = pd.concat([unishape, elmshape, secshape], join='outer', ignore_index=True, sort=False) # Merge DFs
sdshape.head(10)

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,GISJOIN,Shape_Leng,Shape_Area,geometry,ELSDLEA,SCSDLEA
0,06,00001,0600001,Acton-Agua Dulce Unified School District,00,KG,12,G5420,None,E,5.551751e+08,182565.0,+34.4509057,-118.2276141,G06000001,110175.647120,5.553578e+08,"POLYGON ((-1992050.8324 -117279.0507999994, -1...",NaN,NaN
1,06,00009,0600009,Cuyama Joint Unified School District,00,KG,12,G5420,None,E,1.864349e+09,20054388.0,+34.8301010,-119.5209313,G06000009,308308.471429,1.884403e+09,"POLYGON ((-2103814.7531 -21629.76970000006, -2...",NaN,NaN
2,06,00011,0600011,Fort Sage Unified School District,00,KG,12,G5420,None,E,6.826892e+08,5404559.0,+39.9744995,-120.0901010,G06000011,160781.482128,6.880937e+08,POLYGON ((-2004728.815400001 552450.2798999995...,NaN,NaN
3,06,00013,0600013,Rocklin Unified School District,00,KG,12,G5420,None,E,4.871336e+07,99188.0,+38.8080407,-121.2503648,G06000013,33017.520300,4.881255e+07,POLYGON ((-2144286.406199999 436434.3016999997...,NaN,NaN
4,06,00014,0600014,Hesperia Unified School District,00,KG,12,G5420,None,E,3.582757e+08,4180205.0,+34.3616273,-117.3369800,G06000014,78801.356019,3.624559e+08,POLYGON ((-1921037.820800001 -123437.125099999...,NaN,NaN
5,06,00016,0600016,Upland Unified School District,00,KG,12,G5420,None,E,6.788848e+07,502750.0,+34.1463165,-117.6585233,G06000016,42826.143777,6.839124e+07,POLYGON ((-1970528.456900001 -155252.044500000...,NaN,NaN
6,06,00018,0600018,Mountain Valley Unified School District,00,KG,12,G5420,None,E,1.630741e+09,99155.0,+40.5422540,-123.2406802,G06000018,211362.608546,1.630841e+09,"POLYGON ((-2259981.4526 686641.2267000005, -22...",NaN,NaN
7,06,00019,0600019,Dublin Unified School District,00,KG,12,G5420,None,E,3.542607e+07,0.0,+37.7153770,-121.9104967,G06000019,37799.106445,3.542606e+07,"(POLYGON ((-2235995.2443 329544.7997999992, -2...",NaN,NaN
8,06,00020,0600020,Pleasanton Unified School District,00,KG,12,G5420,None,E,1.024609e+08,391116.0,+37.6649739,-121.8867095,G06000020,58631.121814,1.028520e+08,"POLYGON ((-2229697.0096 327679.7281999998, -22...",NaN,NaN
9,06,00021,0600021,Sunol Glen Unified School District,00,KG,12,G5420,None,E,2.443230e+08,2809642.0,+37.5511338,-121.8054596,G06000021,76479.473027,2.471326e+08,POLYGON ((-2228584.443600001 317234.8045000006...,NaN,NaN


In [ ]:
# district.head(20)

#'district' has polygons of the districts.
# list(district)

In [ ]:
# seconddata.head(20)

In [ ]:
# thirdfdata.head()

In [ ]:
# with open(dir_prefix + 'data_management/data/old/charters_parsed_03-04_no-text_SMALL.csv', 'r') as csv_file:
#     csv_reader = csv.reader(csv_file)
    
#     #for line in csv_reader:
#         #print(line)

## Define helper functions

In [5]:
def refresh_data():
    '''Loads data from file, undoing any bounding boxes, etc. 
    Great for making a fresh start when plotting a new area.
    
    Also merges geometric data from shapefile DF into full school district DF.
    To do this, first creates a common identifier across these DFs by modifying district 'Qualifying Name' 
    so it lines up with the district shapefile.'''
    
    global charterDF, district_df, publicDF, sdshape, gjson
    
    # Import data:
    #gc.disable()  
    #Note for Jaren, gc is not defined, what is it supposed be. I believe it was about convert_df but it is not working
    charterDF = pd.read_csv(charterfile, low_memory=False) # Charter school data
    #publicDF = pd.read_csv(publicfile, low_memory=False) # Public school data
    #publicDF = convert_df(publicDF) # Make memory-efficient
    districtDF = pd.read_csv(districtfile, encoding='latin-1', low_memory=False) # School district data
    #gc.enable()
    
    # Merge sdshape with districtDF:
    districtDF['NAME'] = districtDF['Qualifying Name'].apply(lambda x: ",".join(x.split(",")[:-1]).strip())
    district_df = pd.merge(districtDF, sdshape, on="NAME") 
    district_df = gpd.GeoDataFrame(district_df)
    district_df = district_df.apply(pd.to_numeric, errors ='ignore')
    district_df.crs = sdshape.crs
    gjson = district_df.to_json()
    #list district_df gives you the column names. 
    
    '''NOTES:
    #district_df and charter_df are geodataframes, publicdf is neither it is some kind of data
    #district df both has the polygons and the total white percentage stuff
    #charterdf has points!
    #district_df.head()
    #whites = district_df[['% Total Population: White Alone']]
    #whites.dtypes
    #type(names)
    #names = names.apply(pd.to_numeric)
    #names.dtypes
    # districtchanged['style'] = [
    #     {'fillColor': '#ff0000', 'weight': 2, 'color': 'black'},
    #     {'fillColor': '#00ff00', 'weight': 2, 'color': 'black'},
    #     {'fillColor': '#0000ff', 'weight': 2, 'color': 'black'}
    # ]
    '''
    
    # Create geometry for charter schools:
    charterDF['geometry'] = charterDF[['LONGCODE', 'LATCODE']].apply(lambda row: Point(row["LONGCODE"], row["LATCODE"]), axis=1) # Use GeoPandas to create Point class for each school
    charterDF = gpd.GeoDataFrame(charterDF)
    charterDF.crs = {'init' :'epsg:4326'}
    charterDF = charterDF.to_crs(sdshape.crs)
    #charterdf has ideology information. 
    
    
    district_df.crs = sdshape.crs
    gjson = district_df.to_json()

In [6]:
def convert_df(df):
    """Makes a Pandas DataFrame more memory-efficient through intelligent use of Pandas data types: 
    specifically, by storing columns with repetitive Python strings not with the object dtype for unique values 
    (entirely stored in memory) but as categoricals, which are represented by repeated integer values. This is a 
    net gain in memory when the reduced memory size of the category type outweighs the added memory cost of storing 
    one more thing. As such, this function checks the degree of redundancy for a given column before converting it."""
    
    converted_df = pd.DataFrame() # Initialize DF for memory-efficient storage of strings (object types)
    # TO DO: Infer dtypes of df
    df_obj = df.select_dtypes(include=['object']).copy() # Filter to only those columns of object data type

    for col in df.columns: 
        if col in df_obj: 
            num_unique_values = len(df_obj[col].unique())
            num_total_values = len(df_obj[col])
            if (num_unique_values / num_total_values) < 0.5: # Only convert data types if at least half of values are duplicates
                converted_df.loc[:,col] = df[col].astype('category') # Store these columns as dtype "category"
            else: 
                converted_df.loc[:,col] = df[col]
        else:    
            converted_df.loc[:,col] = df[col]
                      
    converted_df.select_dtypes(include=['float']).apply(pd.to_numeric,downcast='float')
    converted_df.select_dtypes(include=['int']).apply(pd.to_numeric,downcast='signed')
    
    return converted_df

In [7]:
sdshape.crs

{'proj': 'aea',
 'lat_1': 29.5,
 'lat_2': 45.5,
 'lat_0': 37.5,
 'lon_0': -96,
 'x_0': 0,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'm',
 'no_defs': True}

In [8]:
refresh_data()
#this takes like a minute or two, be careful. what you do before might be lost.

In [ ]:
# dff = pd.DataFrame({'ints': ['3', '5'], 'Words Ol': ['Kobe', 'Bryant']})
# dff = dff.apply(pd.to_numeric, errors='ignore')
# #type(dff)
# #dff = dff.where(dff.ints != 5)
# dff = dff[(dff.iloc[:, 6] = 6)]
#            #.str.contains('rya'))]
# dff

In [9]:
district_df.head()

,FIPS,Name of Area,Qualifying Name,State/U.S.-Abbreviation (USPS),Summary Level,File Identification,Logical Record Number,State (FIPS),School District (Elementary),School District (Secondary),...,ALAND,AWATER,INTPTLAT,INTPTLON,GISJOIN,Shape_Leng,Shape_Area,geometry,ELSDLEA,SCSDLEA
0,100001.0,"Fort Rucker School District, Alabama","Fort Rucker School District, Alabama",al,970,ACSSF,8710,1,NaN,NaN,...,2.330600e+08,2735224.0,31.409737,-85.745807,G01000001,97610.669919,2.357952e+08,POLYGON ((964471.7861000001 -619835.7846999997...,NaN,NaN
1,100003.0,"Maxwell AFB School District, Alabama","Maxwell AFB School District, Alabama",al,970,ACSSF,8711,1,NaN,NaN,...,8.476761e+06,566857.0,32.380944,-86.363749,G01000003,14236.382240,9.043613e+06,POLYGON ((900511.9332999997 -524987.7790999999...,NaN,NaN
2,100005.0,"Albertville City School District, Alabama","Albertville City School District, Alabama",al,970,ACSSF,8712,1,NaN,NaN,...,6.873181e+07,258708.0,34.263130,-86.210660,G01000005,121162.081906,6.899051e+07,(POLYGON ((889477.8247999996 -317969.414699999...,NaN,NaN
3,100006.0,"Marshall County School District, Alabama","Marshall County School District, Alabama",al,970,ACSSF,8713,1,NaN,NaN,...,1.364101e+09,7842604.0,34.867195,-89.603103,G28002850,301456.169769,1.371943e+09,POLYGON ((602420.1983999982 -276312.5311000003...,NaN,NaN
4,100006.0,"Marshall County School District, Alabama","Marshall County School District, Alabama",al,970,ACSSF,8713,1,NaN,NaN,...,1.267143e+09,103647100.0,34.373997,-86.344270,G01000006,629389.578406,1.370790e+09,"(POLYGON ((897668.364599999 -324489.5702, 8976...",NaN,NaN


In [10]:
# print (type(district_df))
#district_df and charter_df are geodataframes, publicdf is neither it is some kind of data
#district df both has the polygons and the total white percentage stuff
#charterdf has points!
#district_df.head()
#whites = district_df[['% Total Population: White Alone']]
#whites.dtypes

#creation of district data easy again
districtchanged = district_df.apply(pd.to_numeric, errors ='ignore')
#type(names)
#names = names.apply(pd.to_numeric)
#names.dtypes
#list(districtchanged)
#need to get only california(or even better sf data)
#districtchanged = district_df
#districtchanged = districtchanged[(districtchanged.iloc[: , 3] == 'ca')]    #İlk üc columna bakıo. 

districtchangedu = districtchanged[['FIPS','Name of Area','% Total Population: White Alone','Shape_Leng','Shape_Area','geometry']]
#districtchangedu = districtchangedu.loc[983: 1500, :]
districtchanged = districtchangedu
# districtchanged['style'] = [
#     {'fillColor': '#ff0000', 'weight': 2, 'color': 'black'},
#     {'fillColor': '#00ff00', 'weight': 2, 'color': 'black'},
#     {'fillColor': '#0000ff', 'weight': 2, 'color': 'black'}
# ]
districtchanged



#try FIPS = 6

,FIPS,Name of Area,% Total Population: White Alone,Shape_Leng,Shape_Area,geometry
0,100001.0,"Fort Rucker School District, Alabama",79.90,97610.669919,2.357952e+08,POLYGON ((964471.7861000001 -619835.7846999997...
1,100003.0,"Maxwell AFB School District, Alabama",64.04,14236.382240,9.043613e+06,POLYGON ((900511.9332999997 -524987.7790999999...
2,100005.0,"Albertville City School District, Alabama",91.85,121162.081906,6.899051e+07,(POLYGON ((889477.8247999996 -317969.414699999...
3,100006.0,"Marshall County School District, Alabama",94.18,301456.169769,1.371943e+09,POLYGON ((602420.1983999982 -276312.5311000003...
4,100006.0,"Marshall County School District, Alabama",94.18,629389.578406,1.370790e+09,"(POLYGON ((897668.364599999 -324489.5702, 8976..."
5,100006.0,"Marshall County School District, Alabama",94.18,119941.110936,8.810379e+08,POLYGON ((683498.7168000005 -30136.98530000076...
6,100006.0,"Marshall County School District, Alabama",94.18,125282.440985,8.084751e+08,"POLYGON ((1304935.894200001 382478.6041999999,..."
7,100006.0,"Marshall County School District, Alabama",94.18,187604.621103,9.742443e+08,POLYGON ((834317.7391999997 -160248.6121999994...
8,2103810.0,"Marshall County School District, Kentucky",98.00,301456.169769,1.371943e+09,POLYGON ((602420.1983999982 -276312.5311000003...
9,2103810.0,"Marshall County School District, Kentucky",98.00,629389.578406,1.370790e+09,"(POLYGON ((897668.364599999 -324489.5702, 8976..."


In [11]:
#list(districtcha)
#X X X map charterdf data on districtDF map 
#just charterdf colorfully on to a map. 

#the sooner you get to the tableou ohoto jaren sent, its better. work on getting that.
#for districts color with %total population, white alone. 

##Jaren asked, all 3 district files working and make the folium work. 

# CREATION OF CHARTERDF EASY DATAFRAME
charterDF
charterchanged = charterDF.apply(pd.to_numeric, errors ='ignore')
#comment the next line to draw for all states.
# charterchanged = charterchanged[(charterchanged.iloc[: , 0] == 'CALIFORNIA')]
charterchanged
charterchanged = charterchanged[['LEA_NAME','SCH_NAME','geometry']]
charterchanged
charterchanged5 = charterchanged.head(5)
charterchanged
charterchanged['latitude'] = (charterchanged.geometry.x)
charterchanged['longitude'] = (charterchanged.geometry.y)
charterchanged
charterdropped = charterchanged.dropna()
charterdropped

,LEA_NAME,SCH_NAME,geometry,latitude,longitude
0,Lower Kuskokwim School District,Ayaprun Elitnaurvik,POINT (-3698126.569546513 3859820.829485077),-3.698127e+06,3.859821e+06
1,Ketchikan Gateway Borough School District,Ketchikan Charter School,POINT (-2328109.419412659 2408321.007984187),-2.328109e+06,2.408321e+06
2,Ketchikan Gateway Borough School District,Tongass School of Arts and Sciences Charter Sc...,POINT (-2328109.419412659 2408321.007984187),-2.328109e+06,2.408321e+06
3,Anchorage School District,Aquarian Charter School,POINT (-3093389.285215473 3467742.156542414),-3.093389e+06,3.467742e+06
4,Anchorage School District,Family Partnership Charter School,POINT (-3090998.731611759 3466887.320557346),-3.090999e+06,3.466887e+06
5,Anchorage School District,Winterberry School,POINT (-3086881.254226208 3463840.630775834),-3.086881e+06,3.463841e+06
6,Anchorage School District,Eagle Academy Charter School,POINT (-3069435.014155697 3467383.870144491),-3.069435e+06,3.467384e+06
7,Anchorage School District,Frontier Charter School,POINT (-3091945.754119765 3467116.654106261),-3.091946e+06,3.467117e+06
8,Anchorage School District,Highland Tech High Charter School,POINT (-3086095.385439134 3463283.728068264),-3.086095e+06,3.463284e+06
9,Anchorage School District,Rilke Schule Charter School,POINT (-3093326.880683478 3465443.497560237),-3.093327e+06,3.465443e+06


In [12]:

# chartmap = folium.Map([37.7556, -122.4399], zoom_start = 7)
# fg=folium.FeatureGroup(name="Charter schools")
# for index, row in charterchanged.iterrows():
#     fg.add_child(folium.Marker(location=[lat,lon],popup=(folium.Popup(name)),icon=folium.Icon(color=color(elev),icon_color='green')))
# charterchanged.crs = sdshape.crs
# latlong = [xy for xy in charterchanged['geometry']]
# latlong

#IGNORE THIS CODE
charterchanged5.crs = sdshape.crs
charterchanged5 = charterchanged5.to_crs(epsg='4326')
cjson = charterchanged5.to_json()
print(cjson)
#charterchanged5.plot(marker='o', color='blue', markersize=0.5)
# print(charterchanged.crs)
# charterjson = charterchanged.to_json()
# #folium.GeoJson(charterchanged).add_to(chartmap)
# points = folium.features.GeoJson(charterjson)
# # my_marker_cluster = folium.MarkerCluster().add_to(chartmap) 
# # for ix, row in deaths.iterrows(): 
# #     text = "Name: " + row['SCH_NAME'] + "<br>" + "Location: " + str(row['LEA_NAME']) 
# #     popup = folium.Popup(IFrame(text, width=300, height=100)) 
# #     folium.Marker(location = [row['Latitude'],row['Longitude']], popup=popup).add_to(my_marker_cluster)
# charterchanged5['latitude'] = (charterchanged5.geometry.x)
# charterchanged5['longitude'] = (charterchanged5.geometry.y)
# locations  = charterchanged5[['latitude', 'longitude']]
# locationlist = locations.values.tolist()
# print(locationlist)
# charterchanged5
# cmap = folium.Map(location=[37.7556, -122.4399], zoom_start = 7)
# for point in range(0, len(locationlist)):
#     folium.Marker(locationlist[point], popup='is working?')
# cmap
#     #chartmap.add
# chartmap.add_child(points)
# chartmap.save(os.path.join(dir_prefix, 'geospatial/shapefiles/charterschools.html'))
# chartmap

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"LEA_NAME": "Lower Kuskokwim School District", "SCH_NAME": "Ayaprun Elitnaurvik"}, "geometry": {"type": "Point", "coordinates": [-161.76519399999998, 60.796131]}}, {"id": "1", "type": "Feature", "properties": {"LEA_NAME": "Ketchikan Gateway Borough School District", "SCH_NAME": "Ketchikan Charter School"}, "geometry": {"type": "Point", "coordinates": [-131.64119100000002, 55.34700099999997]}}, {"id": "2", "type": "Feature", "properties": {"LEA_NAME": "Ketchikan Gateway Borough School District", "SCH_NAME": "Tongass School of Arts and Sciences Charter School"}, "geometry": {"type": "Point", "coordinates": [-131.64119100000002, 55.34700099999997]}}, {"id": "3", "type": "Feature", "properties": {"LEA_NAME": "Anchorage School District", "SCH_NAME": "Aquarian Charter School"}, "geometry": {"type": "Point", "coordinates": [-149.91687199999996, 61.19240699999999]}}, {"id": "4", "type": "Feature", "propert

In [ ]:
# charterchanged.plot()
# plt.show()
# cjson = charterchanged5.to_json()
# mapa = folium.Map([37.7556, -122.4399], zoom_start = 7)

# points = folium.features.GeoJson(cjson)

# mapa.add_child(points)
# mapa
#charterchanged5['lat'] = charterchanged5.apply(lambda p: p.y)
# chartmap = folium.Map([37.7556, -122.4399], zoom_start = 7)
# cjson = charterchanged5.to_json()
# points = folium.features.GeoJson(cjson)

# chartmap.add_child(points)
# chartmap.save(os.path.join(dir_prefix, 'geospatial/shapefiles/charterschools.html'))
# chartmap

In [13]:
charterdropped.crs = sdshape.crs
charterdropped = charterdropped.to_crs(epsg='4326')
# when run twice we have problem
cjson = charterdropped.to_json()
mapa = folium.Map([37.7556, -120.4399], zoom_start = 7)

points = folium.features.GeoJson(cjson)
#what is points
mapa.add_child(points)
mapa.save(os.path.join(dir_prefix, 'geospatial/shapefiles/charterschools.html'))
mapa

In [14]:
#DONT RUN, NOT WORKING
#received memory error, probably cause there is too much data for folium
mappo = folium.Map([37.7556, -122.4399], zoom_start = 8)
#threshold_scale = spli
def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    if classifier == 'Fisher_Jenks':
        threshold_scale = ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    #    if classifier == 'Equal_Interval':
    #         threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    #    if classifier == 'Quantiles':
    #         threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    #might need to to crs district
    
    #gdf_w = gdf.to_crs({'init': 'epsg:4326'})
    gdf.crs = {'init' :'epsg:4326'}
    gdfw = gdf.to_crs(gdf.crs)
    mapobj.choropleth(geo_data = gdf, name='choropleth', data = gdf, columns = [id_field, value_field], key_on = 'feature.id', \
                     fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity, threshold_scale = threshold_scale)
    return mapobj
mappo = add_choropleth(mappo, districtchanged, 'Population 25 Years and Over:', '% Total Population: White Alone')
mappo

/opt/conda/lib/python3.6/site-packages/pysal/esda/mapclassify.py:267: RuntimeWarning: invalid value encountered in greater
  binIds += (x > l) * (x <= r) * k
/opt/conda/lib/python3.6/site-packages/pysal/esda/mapclassify.py:267: RuntimeWarning: invalid value encountered in less_equal
  binIds += (x > l) * (x <= r) * k
/opt/conda/lib/python3.6/site-packages/numpy/lib/function_base.py:4016: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


KeyError: 'Population 25 Years and Over:'

In [15]:
#districtchanged.crs = {'init': 'epsg:4326'}
districtchanged.crs = district_df.crs
print(districtchanged.crs)
type(districtchanged)
#strr = districtchanged.crs['init'].split(':')[1]
#pyepsg.get(strr)
gjson = districtchanged.to_json()
# districtchanged.plot()
# f, ax = plt.subplots(1, figsize=(10, 6))
# ax.set_title('White percentage')
# districtchanged.plot(column='% Total Population: White Alone', scheme='fisher_jenks', k=7, 
#                          cmap=plt.cm.Blues, legend=True, ax=ax)
# ax.set_axis_off()
# plt.axis('equal');

{'proj': 'aea', 'lat_1': 29.5, 'lat_2': 45.5, 'lat_0': 37.5, 'lon_0': -96, 'x_0': 0, 'y_0': 0, 'datum': 'NAD83', 'units': 'm', 'no_defs': True}


In [ ]:
m = folium.Map([37.7556, -122.4399], zoom_start = 7)
m.choropleth(
    geo_data=districtchanged,
    data=districtchanged,
    columns=['FIPS', '% Total Population: White Alone'],
    key_on='feature.properties.FIPS',
    legend_name='White people', 
    fill_color='YlGn',
    fill_opacity=0.4,
    highlight=True)
m.save(os.path.join(dir_prefix, 'geospatial/shapefiles/districtsca.html'))
m
#causes problems because I believe there are districts with no polygon or something else thats missing. 
#Thats why probably matplotlib can only draw some parts of it

In [16]:
# charterDF = charterDF.to_crs(district.crs)

mappo = folium.Map([37.7556, -122.4399], zoom_start = 7)
#crss = {'init' : 'epsg:4326'}
# charterDF.crs = {'init' :'epsg:4326'}
# charterDF = charterDF.to_crs(district.crs)
#districtchanged.crs = {'init' : 'epsg:4326'}
#districtchanged = districtchanged.to_crs(district.crs)
#districtchanged = districtchanged.to_crs(crss)
#folium.GeoJson(gjson).add_to(mappo)
mappo.choropleth(geo_data = gjson, name='choropleth', data = districtchanged, fill_color = 'YlOrRd',fill_opacity = 0.6, line_opacity = 0.2) 
mappo.save(os.path.join(dir_prefix, 'geospatial/data/geopandas2.html'))
#columns = [id_field, value_field], key_on = 'feature.id', fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity)
mappo
#causes problems because I believe there are districts with no polygon or something else thats missing. 
#Thats why probably matplotlib can only draw some parts of it

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#MAIN METHOD THAT RETURNS CHOROPLETH MAP WITH POINTS

charterdropped.crs = sdshape.crs
charterdropped = charterdropped.to_crs(epsg='4326')
cjson = charterdropped.to_json()
mappo = folium.Map([37.7556, -122.4399], zoom_start = 7)
#mappo.choropleth(
#    geo_data=districtchanged,
#    data=districtchanged,
#    columns=['FIPS', '% Total Population: White Alone'],
#    key_on='feature.properties.FIPS',
#    legend_name='White people', 
#    fill_color='YlGn',
#    fill_opacity=0.4,
#    highlight=True) 
#points = folium.features.GeoJson(cjson)
#mappo.add_child(points)
#mappo.save(os.path.join(dir_prefix, 'geospatial/data/charterschoolswithdistricts.html'))

#mappo



marker_cluster = folium.plugins.MarkerCluster().add_to(mappo)
for each in charterdropped[0:len(charterdropped)].iterrows():
    folium.Marker(
            location=[each[1]['latitude'], each[1]['longitude']],
            popup=folium.Popup(
                            folium.IFrame(
                                html=each[1]['SCH_NAME']
                                
                            )
            ).add_to(marker_cluster)
    
mappo.save('geospatial/data/clustercharterschoolswithdistricts.html')
mappo



In [17]:
#MAIN METHOD THAT RETURNS CHOROPLETH MAP WITH POINTS

charterdropped.crs = sdshape.crs
charterdropped = charterdropped.to_crs(epsg='4326')
cjson = charterdropped.to_json()
mappo = folium.Map([37.7556, -122.4399], zoom_start = 7)

mappo.choropleth(
    geo_data=districtchanged,
    data=districtchanged,
    columns=['FIPS', '% Total Population: White Alone'],
    key_on='feature.properties.FIPS',
    legend_name='White people', 
    fill_color='YlGn',
    fill_opacity=0.4,
    highlight=True) 
points = folium.features.GeoJson(cjson)
mappo.add_child(points)


#Cluster Method

marker_cluster = MarkerCluster().add_to(mappo)
for each in charterdropped[0:len(charterdropped)].iterrows():
    folium.Marker(
            location=[each[1]['latitude'], each[1]['longitude']],
            popup=folium.Popup(
                            folium.IFrame(
                                html=each[1]['SCH_NAME'])
                
                                         
            ).add_to(marker_cluster))
    


In [ ]:
mappo.save('clustercharterschoolswithdistricts.html')
IFrame('clustercharterschoolswithdistricts.html')

In [ ]:
# Should probably save the above chunk this way. But wait until is actually maps successfully above. 

# mappo.save(os.path.join(dir_prefix, 'geospatial/data/charterschoolswithdistricts.html'))
# Iframe('geospatial/data/charterschoolswithdistricts.html', width=700, height=400)


In [ ]:
# def createBoundingBox(boundingBox):
#     '''Pre-requisite function for creating maps within a bounded area or "boundingBox".
#     This function accomplishes this by limiting the relevant DataFrames to only those data points
#     located within the bounded area defined in the two points indicated in the function call. 
#     This allows the plot() function to map the entire DF (which it is designed to do).
    
#     Input is a list of four numbers: [upperleft longitude, upperleft latitude, lowerleft longitude, lowerleft latitude].
#     No direct output--modifies DFs.'''
    
#     global publicDF
#     global charterDF
#     global district_df
    
#     ULLON = boundingBox[0]
#     ULLAT = boundingBox[1]
#     LRLON = boundingBox[2]
#     LRLAT = boundingBox[3]
    
#     publicDF = publicDF[publicDF['LON1516']>ULLON]
#     publicDF= publicDF[publicDF['LAT1516']<ULLAT]
#     publicDF = publicDF[publicDF['LON1516']<LRLON]
#     publicDF = publicDF[publicDF['LAT1516']>LRLAT]
#     charterDF = charterDF[charterDF['LONGCODE']>ULLON]
#     charterDF = charterDF[charterDF['LATCODE']<ULLAT]
#     charterDF = charterDF[charterDF['LONGCODE']<LRLON]
#     charterDF = charterDF[charterDF['LATCODE']>LRLAT]

#     validDistricts = "" # String of all districts within bounded area, appended one at a time
    
#     # Iterate over publicDF, adding each valid district into string validDistricts:
#     for index, row in publicDF.iterrows():
#         try:
#             validDistricts += str(int(publicDF["LEAID"][index])) + " " # Grab LEAIDs for valid districts, indexing by column and then row
#         except Exception as e:
#             continue

#     dropList = []
    
#     for index, row in district_df.iterrows():
#         try:
#             if ((str(int(district_df["FIPS"][index]))) not in validDistricts):
#                 dropList.append(index)
#         except Exception as e:
#             continue
            
#     district_df = district_df.drop(dropList) # Drop districts not in bounding box
    
#     return

In [ ]:
# def plot(district_col, whole=True, state="", ab=""):
#     '''Maps school districts in given area as chloropleth map shaded by variable district_col, 
#     showing points for schools based on latitude/longitude and ess_lean. 
#     Points are shaded such that blue is lower in ess_lean and red is higher.
    
#     The only required parameter is district_col, the variable/column by which to shade school districts. 
#     If bounding boxes are being used, ignore the other parameters below. 
#     Example function call: 
#     plot('Median household income (In 2014 Inflation Adjusted Dollars')
    
#     Other parameters:
#     whole = whether or not to map entire country (Boolean). If False, need BOTH of the following:
#     state = whether to map a particular state
#     ab = single state abbreviation, e.g. FL
    
#     Leaving 'whole' as False and giving a state will plot that state only;
#     Leaving 'whole' as False and giving no state will plot the whole US.
    
#     Note on working with these functions:
#     To use the single-state functionality of this function AFTER creating a bounding box, 
#     rerun the notebook in order to recreate the full dataframes.'''
    
    
#     figsize = (14, 7) if whole else (10,7) # Define scale for US (if whole) or single state (if not)
#     fig, ax = plt.subplots(figsize=figsize)
#     tempdistrict_df = district_df # Make temporary DF for within function
    
#     # If mapping just one state, define state abbreviation column name:
#     if not whole:
#         tempdistrict_df = tempdistrict_df[tempdistrict_df['State/U.S.-Abbreviation (USPS)'] == ab]
        
#     tempdistrict_df = tempdistrict_df[[district_col, 'geometry']].dropna() # Drop if no coordinates
#     tempdistrict_df[district_col] = tempdistrict_df[district_col].apply(pd.to_numeric,downcast='float') # Convert to most efficient float type
    
#     # Plot layers of map:
#     tempdistrict_df.plot(ax = ax, column=district_col, cmap='PiYG', linewidth=0, legend=True) # Plot shaded areas
#     charterDF[whole | (charterDF['STATENAME'] == state)].plot(ax = ax, cmap="coolwarm", column="ess_lean", markersize=20, linewidth=0, legend=True) # Plot points
#     # Note: cmap = shading style (e.g., PiYG = pink to green)
    
#     ax.legend()
#     plt.show()
    
#     return

## Plotting

In [ ]:
refresh_data() # Load in data!

In [ ]:
# # Example of using mapping function:
# bayArea = [-123.26482826886195, 38.66436310541212, -121.06756264386195, 37.12821658249273]
# createBoundingBox(bayArea)
# district_df

In [ ]:
refresh_data() # Refresh data and undo Bay Area bounding box

# Remove outliers using ess_lean:
charterDF['ess_lean'] = charterDF['ess_strength'] - charterDF['prog_strength']
charterDF = charterDF.dropna(subset=["ess_lean"])
high = charterDF['ess_lean'].quantile(.98) # Find upper limit of ess_lean (a number)
low = charterDF['ess_lean'].quantile(.02)
charterDF = charterDF[(charterDF['ess_lean'] < high) & (charterDF['ess_lean'] > low)]

In [ ]:
plot('% Total Population: White Alone', False, 'CALIFORNIA', 'ca') 

In [ ]:
plot('% Total Population: White Alone')

In [ ]:
plot('% Total Population: White Alone')

In [ ]:
plot('Total Population: Foreign Born: Not a Citizen')